# RQ1

In [184]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 200)

In [189]:
import pandas as pd
import os

def get_path_apgd(path_dir_compile):
    path_list = []
    if os.path.isdir(path_dir_compile):
        for root, dirs, files in os.walk(path_dir_compile, topdown=True):
            for file in files:
                file_absolute_path = os.path.join(root, file)
                if file_absolute_path.endswith('.csv') and 'apfd' in file_absolute_path:
                    path_list.append(file_absolute_path)
    return path_list

def check_attack(path):
    attack_check_list = ['pgdattack', 'nodeembeddingattack', 'randomattack', 'dice', 'minmax']
    for i in attack_check_list:
        if i in path:
            return True
    return False


def check_weight_stacking_voting(s):
    if 'weight' in s:
        return 1
    if 'stacking' in s:
        return 1
    if 'voting' in s:
        return 1
    return 0

path_list = get_path_apgd('/Users/yinghua.li/Documents/Pycharm/NodeRank/res')
path_list = sorted([i for i in path_list if not check_attack(i)])
# path_list.append('/Users/yinghua.li/Documents/Pycharm/NodeRank/res/apfd_citeseer_gcn.csv')
path_list = sorted(path_list)
data_list = []
for path in path_list:
    df = pd.read_csv(path, header=None, names=['subject', 'apfd'])
    data_list.append(df)
df = pd.concat(data_list, ignore_index=True)
df['data'] = [i.split('_')[0] for i in df['subject']] 
df['gnn'] = [i.split('_')[1] for i in df['subject']] 
df['method'] = [i.split('_')[2] for i in df['subject']]

dic = {'gat': 'GAT', 'gcn': 'GCN', 'tagcn': 'TAGCN', 'graphsage': 'GraphSAGE', 'xgb': 'XGB', 'lgb': 'LGB', 'rf': 'RF', 
       'lr': 'LR', 'mutation': 'Mutation', 'fusion': 'NodeRank',
       'deepGini': 'DeepGini', 'leastConfidence': 'Least Confidence', 'margin': 'Margin', 'random': 'Random'}
dic_data = {'citeseer': 'CiteSeer', 'cora': 'Cora', 'lastfm': 'LastFM', 'pubmed': 'PubMed'}
df['data'] = [dic_data[i] for i in df['data']]

df['gnn'] = [dic[i] for i in df['gnn']]
df['method'] = [dic[i] for i in df['method']]
df = df[(df['method']!='XGB')&(df['method']!='LGB')&
        (df['method']!='RF')&(df['method']!='LR')&
        (df['method']!='Mutation')]

df['check'] = df['subject'].apply(check_weight_stacking_voting)
df = df[df['check']==0]
del df['check']

approach_list = ['Random', 'Margin','Least Confidence', 'DeepGini', 'NodeRank']
data_list = ['CiteSeer', 'Cora', 'LastFM', 'PubMed']
gnn_list = ['GAT', 'GCN', 'GraphSAGE', 'TAGCN']

res_list = []
for method in approach_list:
    tmp_list = [method]
    for data in data_list:
        for gnn in gnn_list:
            apfd = df[(df['method']==method) & (df['data']==data) & (df['gnn']==gnn)]['apfd'].values[0]
            apfd = str(round(apfd, 3))
            apfd = apfd +'0'* (5-len(apfd))
            tmp_list.append(apfd)
    res_list.append(tmp_list)  
    
df_re = pd.DataFrame(res_list, columns=['Method']+['GAT', 'GCN', 'GraphSAGE', 'TAGCN']*4)

df_re.to_excel('table_overleaf/RQ1-1.xlsx', index=False)

df_re

,Method,GAT,GCN,GraphSAGE,TAGCN,GAT,GCN,GraphSAGE,TAGCN,GAT,GCN,GraphSAGE,TAGCN,GAT,GCN,GraphSAGE,TAGCN
0,Random,0.519,0.493,0.493,0.512,0.503,0.479,0.517,0.494,0.501,0.504,0.525,0.500,0.502,0.492,0.493,0.508
1,Margin,0.681,0.673,0.682,0.680,0.744,0.728,0.718,0.735,0.653,0.750,0.772,0.763,0.649,0.600,0.659,0.662
2,Least Confidence,0.675,0.672,0.666,0.673,0.753,0.744,0.735,0.741,0.639,0.737,0.781,0.772,0.662,0.709,0.659,0.658
3,DeepGini,0.638,0.644,0.612,0.628,0.720,0.727,0.710,0.693,0.586,0.692,0.759,0.740,0.638,0.694,0.608,0.615
4,NodeRank,0.718,0.701,0.730,0.723,0.822,0.819,0.819,0.792,0.825,0.830,0.831,0.823,0.790,0.774,0.817,0.812


In [190]:
del df_re['Method']

In [208]:
improvement = []
np_re = df_re.to_numpy().astype(np.float64)
improvement.append(list((np_re[4]-np_re[0]) / np_re[0]))
improvement.append(list((np_re[4]-np_re[1]) / np_re[1]))
improvement.append(list((np_re[4]-np_re[2]) / np_re[2]))
improvement.append(list((np_re[4]-np_re[3]) / np_re[3]))

for i in range(len(improvement)):
    for j in range(len(improvement[i])):
        improvement[i][j] = "%.2f%%" % (improvement[i][j]*100)
        
improvement[0].insert(0, 'Random')
improvement[1].insert(0, 'Margin')
improvement[2].insert(0, 'Least Confidence')
improvement[3].insert(0, 'DeepGini')
improvement.append(['NodeRank']+['-']*16)

df_re_2 = pd.DataFrame(improvement, columns=['Method']+['GAT', 'GCN', 'GraphSAGE', 'TAGCN']*4)
df_re_2.to_excel('table_overleaf/RQ1-2.xlsx', index=False)
df_re_2

,Method,GAT,GCN,GraphSAGE,TAGCN,GAT,GCN,GraphSAGE,TAGCN,GAT,GCN,GraphSAGE,TAGCN,GAT,GCN,GraphSAGE,TAGCN
0,Random,38.34%,42.19%,48.07%,41.21%,63.42%,70.98%,58.41%,60.32%,64.67%,64.68%,58.29%,64.60%,57.37%,57.32%,65.72%,59.84%
1,Margin,5.43%,4.16%,7.04%,6.32%,10.48%,12.50%,14.07%,7.76%,26.34%,10.67%,7.64%,7.86%,21.73%,29.00%,23.98%,22.66%
2,Least Confidence,6.37%,4.32%,9.61%,7.43%,9.16%,10.08%,11.43%,6.88%,29.11%,12.62%,6.40%,6.61%,19.34%,9.17%,23.98%,23.40%
3,DeepGini,12.54%,8.85%,19.28%,15.13%,14.17%,12.65%,15.35%,14.29%,40.78%,19.94%,9.49%,11.22%,23.82%,11.53%,34.37%,32.03%
4,NodeRank,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


# RQ2

In [404]:
import pandas as pd
import os

def get_path_apgd(path_dir_compile):
    path_list = []
    if os.path.isdir(path_dir_compile):
        for root, dirs, files in os.walk(path_dir_compile, topdown=True):
            for file in files:
                file_absolute_path = os.path.join(root, file)
                if file_absolute_path.endswith('.csv') and 'apfd' in file_absolute_path:
                    path_list.append(file_absolute_path)
    return path_list


def check_path(s):
    check_list = ['pgdattack', 'nodeembeddingattack_remove', 'randomattack_add',
                  'randomattack_remove', 'dice', 'minmax', 'nodeembeddingattack_add']
    
    for check in check_list:
        if check in s:
            return True
    
    return False

def check_subject(s):
    check_list = ['xgb', 'lgb', 'rf', 'mutation', 'weight', 'stacking', 'voting', 'lr']
    for i in check_list:
        if i in s:
            return True
    return False


def get_attack(s):
    check_list = ['pgdattack', 'nodeembeddingattack_remove', 'randomattack_add',
                  'randomattack_remove', 'dice', 'minmax', 'nodeembeddingattack_add']
    for i in check_list:
        if i in s:
            return i
    return None
    

dic = {'gat': 'GAT', 'gcn': 'GCN', 'tagcn': 'TAGCN', 'graphsage': 'GraphSAGE', 'xgb': 'XGB', 'lgb': 'LGB', 'rf': 'RF', 
       'lr': 'LR', 'mutation': 'Mutation', 'fusion': 'NodeRank',
       'deepGini': 'DeepGini', 'leastConfidence': 'Least Confidence', 'margin': 'Margin', 'random': 'Random',
        'citeseer': 'CiteSeer', 'cora': 'Cora', 'lastfm': 'LastFM', 'pubmed': 'PubMed', 
      
      'dice': 'DICE', 'pgdattack': 'PGD', 'nodeembeddingattack_remove': 'NEAR', 'randomattack_add': 'RAA',
      'randomattack_remove': 'RAR', 'minmax': 'MMA', 'nodeembeddingattack_add': 'NEAA'
      }
    
    
path_list = get_path_apgd('/Users/yinghua.li/Documents/Pycharm/NodeRank/res')
path_list = [i for i in path_list if check_path(i)]

path_list = sorted(path_list)
data_list = []
for path in path_list:
    df = pd.read_csv(path, header=None, names=['subject', 'apfd'])
    data_list.append(df)
df = pd.concat(data_list, ignore_index=True)
df['check'] = [check_subject(i) for i in df['subject']]
df = df[df['check']==False]
del df['check']

df['approach'] = [i.split('_')[-1] for i in df['subject']]
df['approach'] = [dic[i] for i in df['approach']]
df['attack'] = df['subject'].apply(get_attack)
df['attack'] = [dic[i] for i in df['attack']]
df['data'] = [i.split('_')[0] for i in df['subject']]
df['data'] = [dic[i] for i in df['data']]
df['model'] = [i.split('_')[1] for i in df['subject']]
df['model'] = [dic[i] for i in df['model']]


attack_list = ['DICE', 'MMA', 'NEAA', 'NEAR', 'PGD', 'RAA', 'RAR']
approach_list = ['Random', 'Margin','Least Confidence', 'DeepGini', 'NodeRank']
data_list = ['CiteSeer', 'Cora', 'LastFM', 'PubMed']
gnn_list = ['GAT', 'GCN', 'GraphSAGE', 'TAGCN']

# res_list = []
# for attack in attack_list:
#     for data in data_list:
#         for gnn in gnn_list:
#             tmp_list = [attack, data, gnn]
#             for method in approach_list:
#                 try:
#                     apfd = df[(df['attack']==attack)& (df['approach']==method) & (df['data']==data) & (df['model']==gnn)]['apfd'].values[0]
#                     apfd = str(round(apfd, 3))
#                     apfd = apfd +'0'* (5-len(apfd))
#                     tmp_list.append(apfd)
#                 except:
#                     tmp_list.append(None)
#             res_list.append(tmp_list)

res_list = []
for attack in attack_list:
    for method in approach_list:
        tmp_list = [attack, method]
        for data in data_list:
            for gnn in gnn_list:
                try:
                    apfd = df[(df['attack']==attack)& (df['approach']==method) & (df['data']==data) & (df['model']==gnn)]['apfd'].values[0]
                    apfd = str(round(apfd, 3))
                    apfd = apfd +'0'* (5-len(apfd))
                    tmp_list.append(apfd)
                except:
                    tmp_list.append('-')
        res_list.append(tmp_list)
        

df_re = pd.DataFrame(res_list, columns=['Attack', 'Approach']+['GAT', 'GCN', 'GraphSAGE', 'TAGCN']*4)
df_re = df_re.dropna()

df_re.to_excel('table_overleaf/RQ2-1.xlsx', index=False)

In [534]:
res = []
j = 1
for s, ppdf in df.groupby(['attack']):
    approach_list = ['Random', 'Margin','Least Confidence', 'DeepGini', 'NodeRank']
    for approach in approach_list:
        tmp_res = [s, approach]
        pdf = ppdf[ppdf['approach']==approach]
        gnn_list = ['GAT', 'GCN', 'GraphSAGE', 'TAGCN']

        for gnn in gnn_list:
            tmp_pdf = pdf[pdf['model']==gnn]
            apfd = tmp_pdf['apfd'].mean()
            apfd = str(round(apfd, 3))
            apfd = apfd +'0'* (5-len(apfd))
            tmp_res.append(apfd)
        res.append(tmp_res)
improve_res = []
res_np = np.array(res)[:,2:].astype(np.float64)
for i in range(len(res_np)):
    if i <=4:
        tmp = (res_np[4]-res_np[i]) / res_np[i]
        improve_res.append(tmp)
    elif 4<i<=9:
        tmp = (res_np[9]-res_np[i]) / res_np[i]
        improve_res.append(tmp)
    elif 9<i<=14:
        tmp = (res_np[14]-res_np[i]) / res_np[i]
        improve_res.append(tmp)
    elif 14<i<=19:
        tmp = (res_np[19]-res_np[i]) / res_np[i]
        improve_res.append(tmp)
    elif 19<i<=24:
        tmp = (res_np[24]-res_np[i]) / res_np[i]
        improve_res.append(tmp)
    elif 24<i<=29:
        tmp = (res_np[29]-res_np[i]) / res_np[i]
        improve_res.append(tmp)
    elif 29<i<=34:
        tmp = (res_np[34]-res_np[i]) / res_np[i]
        improve_res.append(tmp)
        
def get_3_value(i):
    i = "%.2f%%" % (i*100)
    i = i +'0'* (5-len(i))
    return i

new_res = []
for i in range(len(improve_res)):
    tmp_new_res = []
    for j in range(len(improve_res[j])):
        if improve_res[i][j] == 0:
            tmp_new_res.append('-')
        else:
            tmp_new_res.append(get_3_value(improve_res[i][j]))
    new_res.append(tmp_new_res)
    
new_res = np.array(new_res)
res = np.array(res)

all_res = np.concatenate((res, new_res), axis=1)

df_rq2 = pd.DataFrame(all_res, columns=['Attack', 'Approach']+['GAT', 'GCN', 'GraphSAGE', 'TAGCN']*2)

df_rq2.to_excel('table_overleaf/RQ2-2.xlsx', index=False)

df_rq2

,Attack,Approach,GAT,GCN,GraphSAGE,TAGCN,GAT,GCN,GraphSAGE,TAGCN
0,DICE,Random,0.508,0.498,0.503,0.508,53.35%,55.02%,57.46%,54.53%
1,DICE,Margin,0.658,0.670,0.693,0.690,18.39%,15.22%,14.29%,13.77%
2,DICE,Least Confidence,0.665,0.696,0.695,0.695,17.14%,10.92%,13.96%,12.95%
3,DICE,DeepGini,0.640,0.673,0.663,0.657,21.72%,14.71%,19.46%,19.48%
4,DICE,NodeRank,0.779,0.772,0.792,0.785,-,-,-,-
5,MMA,Random,0.492,0.487,0.490,0.510,58.74%,58.11%,59.39%,51.96%
6,MMA,Margin,0.701,0.694,0.696,0.703,11.41%,10.95%,12.21%,10.24%
7,MMA,Least Confidence,0.707,0.701,0.699,0.702,10.47%,9.84%,11.73%,10.40%
8,MMA,DeepGini,0.679,0.678,0.663,0.660,15.02%,13.57%,17.80%,17.42%
9,MMA,NodeRank,0.781,0.770,0.781,0.775,-,-,-,-


# RQ3

In [720]:
import pandas as pd
import os

def get_path_apgd(path_dir_compile):
    path_list = []
    if os.path.isdir(path_dir_compile):
        for root, dirs, files in os.walk(path_dir_compile, topdown=True):
            for file in files:
                file_absolute_path = os.path.join(root, file)
                if file_absolute_path.endswith('.csv') and 'apfd' in file_absolute_path:
                    path_list.append(file_absolute_path)
    return path_list


def udf_subject(s):
    check_list = ['xgb', 'lgb', 'rf', 'mutation', 'lr']
    for i in check_list:
        if i in s:
            return True
    return False


def udf_attack(s):
    check_list = ['pgdattack', 'nodeembeddingattack_remove', 'randomattack_add', 'randomattack_flip',
                  'randomattack_remove', 'dice', 'minmax', 'nodeembeddingattack_add']
    for i in check_list:
        if i in s:
            return 'yes'
    return 'no'


def udf_get_attack(s):
    check_list = ['pgdattack', 'nodeembeddingattack_remove', 'randomattack_add', 'randomattack_flip',
                  'randomattack_remove', 'dice', 'minmax', 'nodeembeddingattack_add']
    for i in check_list:
        if i in s:
            return i
    

dic = {'gat': 'GAT', 'gcn': 'GCN', 'tagcn': 'TAGCN', 'graphsage': 'GraphSAGE', 'xgb': 'XGB', 'lgb': 'LGB', 'rf': 'RF', 
       'lr': 'LR', 'mutation': 'Mutation', 'fusion': 'NodeRank',
       'deepGini': 'DeepGini', 'leastConfidence': 'Least Confidence', 'margin': 'Margin', 'random': 'Random',
        'citeseer': 'CiteSeer', 'cora': 'Cora', 'lastfm': 'LastFM', 'pubmed': 'PubMed', 
      
      'dice': 'DICE', 'pgdattack': 'PGD', 'nodeembeddingattack_remove': 'NEAR', 'randomattack_add': 'RAA',
      'randomattack_remove': 'RAR', 'minmax': 'MMA', 'nodeembeddingattack_add': 'NEAA'
      }
    
    
path_list = get_path_apgd('/Users/yinghua.li/Documents/Pycharm/NodeRank/res')

path_list = sorted(path_list)
data_list = []
for path in path_list:
    df = pd.read_csv(path, header=None, names=['subject', 'apfd'])
    data_list.append(df)
df = pd.concat(data_list, ignore_index=True)
df['check'] = df['subject'].apply(udf_subject)
df = df[df['check']==False]
del df['check']
df['data'] = [i.split('_')[0] for i in df['subject']]
df['model'] = [i.split('_')[1] for i in df['subject']]
df['approach'] = [i.split('_')[-1] for i in df['subject']]


df['attack_check'] = df['subject'].apply(udf_attack)

df_attack = df[df['attack_check']=='yes']
df_attack['attack'] = df_attack['subject'].apply(udf_get_attack)
df_clean = df[df['attack_check']=='no']

###############################



<ipython-input-720-59f34562153f>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attack['attack'] = df_attack['subject'].apply(udf_get_attack)


In [721]:
approach_list = list(set(df['approach']))
dic = dict(zip(approach_list, [0]*len(approach_list)))

for sub, pdf in df_clean.groupby(['data', 'model']):
    tmp_dic = dict(zip(pdf['approach'], pdf['apfd']))
    key = max(tmp_dic,key=tmp_dic.get)
    dic[key] += 1
    
print('clean', dic)

df_clean = df_clean.groupby(['approach', 'model']).agg({'apfd': 'mean'}).reset_index()

approach_list = ['random', 'margin', 'leastConfidence', 'deepGini', 'voting', 'stacking', 'fusion', 'weight']
model_list = ['gat', 'gcn', 'graphsage', 'tagcn']

res_list = []
for approach in approach_list:
    tmp_res = [approach]
    for model in model_list:
        tmp_apfd = df_clean[(df_clean['approach']==approach) & (df_clean['model']==model)]['apfd'].values[0]
        tmp_res.append(tmp_apfd)
    res_list.append(tmp_res)
    
df_res = pd.DataFrame(res_list, columns=['approach']+model_list)
df_res.to_excel('table_overleaf/RQ3-1.xlsx', index=False)

clean {'deepGini': 0, 'random': 0, 'voting': 0, 'weight': 9, 'margin': 0, 'leastConfidence': 0, 'stacking': 0, 'fusion': 7}


In [722]:
approach_list = list(set(df['approach']))
dic = dict(zip(approach_list, [0]*len(approach_list)))

for sub, pdf in df_attack.groupby(['data', 'attack', 'model']):
    tmp_dic = dict(zip(pdf['approach'], pdf['apfd']))
    key = max(tmp_dic,key=tmp_dic.get)
    dic[key] += 1
    
print('attack', dic)

df_attack = df_attack.groupby(['approach', 'model']).agg({'apfd': 'mean'}).reset_index()

approach_list = ['random', 'margin', 'leastConfidence', 'deepGini', 'voting', 'stacking', 'fusion', 'weight']
model_list = ['gat', 'gcn', 'graphsage', 'tagcn']

res_list = []
for approach in approach_list:
    tmp_res = [approach]
    for model in model_list:
        tmp_apfd = df_attack[(df_attack['approach']==approach) & (df_attack['model']==model)]['apfd'].values[0]
        tmp_res.append(tmp_apfd)
    res_list.append(tmp_res)
    
df_res = pd.DataFrame(res_list, columns=['approach']+model_list)
df_res.to_excel('table_overleaf/RQ3-2.xlsx', index=False)

attack {'deepGini': 0, 'random': 0, 'voting': 0, 'weight': 68, 'margin': 0, 'leastConfidence': 0, 'stacking': 0, 'fusion': 40}


In [723]:
df_res = pd.read_excel('table_overleaf/RQ3_all.xlsx')

In [725]:
del df_res['approach']

In [733]:
pd.DataFrame(np.around(df_res.to_numpy(),4)).to_excel('table_overleaf/RQ3-tmp.xlsx', index=False)

# RQ4

In [68]:
import pandas as pd
import os

def get_path_apgd(path_dir_compile):
    path_list = []
    if os.path.isdir(path_dir_compile):
        for root, dirs, files in os.walk(path_dir_compile, topdown=True):
            for file in files:
                file_absolute_path = os.path.join(root, file)
                if file_absolute_path.endswith('.csv') and 'apfd' in file_absolute_path:
                    path_list.append(file_absolute_path)
    return path_list


def udf_subject(s):
    check_list = ['xgb', 'lgb', 'rf', 'mutation', 'lr']
    for i in check_list:
        if i in s:
            return True
    return False


def udf_attack(s):
    check_list = ['pgdattack', 'nodeembeddingattack_remove', 'randomattack_add', 'randomattack_flip',
                  'randomattack_remove', 'dice', 'minmax', 'nodeembeddingattack_add']
    for i in check_list:
        if i in s:
            return 'yes'
    return 'no'


def udf_get_attack(s):
    check_list = ['pgdattack', 'nodeembeddingattack_remove', 'randomattack_add', 'randomattack_flip',
                  'randomattack_remove', 'dice', 'minmax', 'nodeembeddingattack_add']
    for i in check_list:
        if i in s:
            return i
    

dic = {'gat': 'GAT', 'gcn': 'GCN', 'tagcn': 'TAGCN', 'graphsage': 'GraphSAGE', 'xgb': 'XGB', 'lgb': 'LGB', 'rf': 'RF', 
       'lr': 'LR', 'mutation': 'Mutation', 'fusion': 'NodeRank',
       'deepGini': 'DeepGini', 'leastConfidence': 'Least Confidence', 'margin': 'Margin', 'random': 'Random',
        'citeseer': 'CiteSeer', 'cora': 'Cora', 'lastfm': 'LastFM', 'pubmed': 'PubMed', 
      
      'dice': 'DICE', 'pgdattack': 'PGD', 'nodeembeddingattack_remove': 'NEAR', 'randomattack_add': 'RAA',
      'randomattack_remove': 'RAR', 'minmax': 'MMA', 'nodeembeddingattack_add': 'NEAA'
      }
    
def get_value(i):
    i = str(round(i, 3))
    i = i +'0'* (5-len(i))
    return i
    
path_list = get_path_apgd('/Users/yinghua.li/Documents/Pycharm/NodeRank/res/features')
path_check = [udf_attack(i) for i in path_list]
path_clean = [path_list[i] for i in range(len(path_list)) if path_check[i]=='no']
path_attack = [path_list[i] for i in range(len(path_list)) if path_check[i]=='yes']
path_clean_name = [i.split('_')[1] for i in path_clean]
path_attack_name = [i.split('_')[1] for i in path_attack]

df_clean = pd.DataFrame([list(pd.read_csv(i, header=None)[0]) for i in path_clean])
df_clean['name'] = path_clean_name
df_clean = df_clean.groupby('name').mean().reset_index().set_index('name').T
df_clean['mean'] = (df_clean['citeseer']+df_clean['cora']+df_clean['lastfm']+df_clean['pubmed'])/4
df_clean['citeseer'] = [get_value(i) for i in df_clean['citeseer']]
df_clean['cora'] = [get_value(i) for i in df_clean['cora']]
df_clean['lastfm'] = [get_value(i) for i in df_clean['lastfm']]
df_clean['pubmed'] = [get_value(i) for i in df_clean['pubmed']]
df_clean['mean'] = [get_value(i) for i in df_clean['mean']]

print(df_clean)
df_clean.to_excel('table_overleaf/RQ4-1.xlsx', index=False)

df_attack = pd.DataFrame([list(pd.read_csv(i, header=None)[0]) for i in path_attack])
df_attack['name'] = path_attack_name
df_attack = df_attack.groupby('name').mean().reset_index().set_index('name').T
df_attack['mean'] = (df_attack['citeseer']+df_attack['cora']+df_attack['lastfm']+df_attack['pubmed'])/4
df_attack['citeseer'] = [get_value(i) for i in df_attack['citeseer']]
df_attack['cora'] = [get_value(i) for i in df_attack['cora']]
df_attack['lastfm'] = [get_value(i) for i in df_attack['lastfm']]
df_attack['pubmed'] = [get_value(i) for i in df_attack['pubmed']]
df_attack['mean'] = [get_value(i) for i in df_attack['mean']]

print(df_attack)
df_attack.to_excel('table_overleaf/RQ4-2.xlsx', index=False)


name citeseer   cora lastfm pubmed   mean
0       0.559  0.606  0.615  0.579  0.590
1       0.622  0.673  0.705  0.628  0.657
2       0.719  0.811  0.828  0.797  0.789
name citeseer   cora lastfm pubmed   mean
0       0.563  0.607  0.608  0.573  0.588
1       0.649  0.684  0.712  0.643  0.672
2       0.735  0.810  0.822  0.789  0.789


# RQ5